# Documentation for module outcomeDL.anon - description of use of functions and justification for their design

## Background and motivation
It's important to anchor anonymization to the NEMA DICOM standard - my early attempts at constructing DICOM file objects failed due to a lack of attention to Type 1 (required) vs Type 2/3 information modules. For anonymizing data for deep learning research, the primary assumption is that patient characteristics are stored externally (such as in a CSV) and mapped via MRN to the DICOM objects. As such, it is not necessary to retain *any* information about the patient in the DICOM files themselves, apart from a new anonymized ID that can serve as the mapper to similarly anonymized information sets in patient characteristics and outcome labels.
Therefore, our anonymization functions should walk through the data elements contained in a DICOM file, and if the data element contains data that is relevant to the object itself (such as Rows or Pixel Spacing attributes for images, or actual contour data for structure sets), then that data element should be retained. If the data element is not directly related to the object itself, but rather provides context (such as referring physician, scanner type, etc), then it should be removed. The caveat to such removal is conscientious handling of Type 1, Type 2, and Type 3 data elements. Type 1 must have a value, so any Type 1 element beyond Patient ID that is slated for removal will need a dummy value inserted. Type 2 must be present but is permitted to be zero-value, so these will be replaced by empty strings. Type 3 elements can be removed completely to no ill effect.

For scope clarity, this module only will anonymize CT, MR, PET, RTSTRUCT, RTDOSE, and RTPLAN. Any other modality passed to the function will be not be output to the destination folder.

### Table A.3-1. CT Image IOD Modules

| IE      | MODULE            | REFERENCE | USAGE |
|---------|-------------------|-----------|---|
| Patient | Patient           | C.7.1.1   | M |
|         | Clinical Trial Subject | C.7.1.3 | U |
| Study   | General Study     | C.7.2.1   | M |
|         | Patient Study     | C.7.2.2   | U |
|         | Clinical Trial Study | C.7.2.3 | U |
| Series  | General Series    | C.7.3.1   | M |
|         | Clinical Trial Series | C.7.3.2 | U |
| Frame of Reference | Frame of Reference | C.7.4.1 | M |
|         | Synchronization   | C.7.4.2   | C - Required if time synch was applied |
| Equipment | General Equipment | C.7.5.1 | M |
| Acquisition | General Acquisition | C.7.10.1 | M |
| Image   | General Image     | C.7.6.1   | M |
|         | General Reference | C.12.4    | U |
|         | Image Plane       | C.7.6.2   | M |
|         | Image Pixel       | C.7.6.3   | M |
|         | Contrast/Bolus    | C.7.6.4   | C - Required if contrast was used |
|         | Device            | C.7.6.12  | U |
|         | Specimen          | C.7.6.22  | U |
|         | CT Image          | C.8.2.1   | M |
|         | Multi-energy CT Image | C.8.2.2 | C - Required if Multi-energy CT Acquisition (0018, 9361) is YES. |
|         | Overlay Plane     | C.9.2     | U |
|         | VOI LUT           | C.11.2    | U |
|         | SOP Common        | C.12.1    | M |
|         | Common Instance Reference | C.12.2 | U |

Legend: M - Mandatory; C - Conditional; U - User Option

### Table A.4-1. MR Image IOD Modules

| IE      | MODULE            | REFERENCE | USAGE |
|---------|-------------------|-----------|---|
| Patient | Patient           | C.7.1.1   | M |
|         | Clinical Trial Subject | C.7.1.3 | U |
| Study   | General Study     | C.7.2.1   | M |
|         | Patient Study     | C.7.2.2   | U |
|         | Clinical Trial Study | C.7.2.3 | U |
| Series  | General Series    | C.7.3.1   | M |
|         | Clinical Trial Series | C.7.3.2 | U |
| Frame of Reference | Frame of Reference | C.7.4.1 | M |
| Equipment | General Equipment | C.7.5.1 | M |
| Acquisition | General Acquisition | C.7.10.1 | M |
| Image   | General Image     | C.7.6.1   | M |
|         | General Reference | C.12.4    | U |
|         | Image Plane       | C.7.6.2   | M |
|         | Image Pixel       | C.7.6.3   | M |
|         | Contrast/Bolus    | C.7.6.4   | C - Required if contrast was used |
|         | Device            | C.7.6.12  | U |
|         | Specimen          | C.7.6.22  | U |
|         | MR Image          | C.8.3.1   | M |
|         | Overlay Plane     | C.9.2     | U |
|         | VOI LUT           | C.11.2    | U |
|         | SOP Common        | C.12.1    | M |
|         | Common Instance Reference | C.12.2 | U |

Legend: M - Mandatory; C - Conditional; U - User Option

### Table A.21.3-1. PET Image IOD Modules

| IE      | MODULE            | REFERENCE | USAGE |
|---------|-------------------|-----------|---|
| Patient | Patient           | C.7.1.1   | M |
|         | Clinical Trial Subject | C.7.1.3 | U |
| Study   | General Study     | C.7.2.1   | M |
|         | Patient Study     | C.7.2.2   | U |
|         | Clinical Trial Study | C.7.2.3 | U |
| Series  | General Series    | C.7.3.1   | M |
|         | Clinical Trial Series | C.7.3.2 | U |
|         | PET Series        | C.8.9.1   | M |
|         | PET Isotope       | C.8.9.2   | M |
|         | PET Multi-gated Acquisition | C.8.9.3 | C - Required if Series Type (0054,1000) Value 1 is GATED. |
|         | NM/PET Patient Orientation | C.8.4.6 | M |
| Frame of Reference | Frame of Reference | C.7.4.1 | M |
|         | Synchronization   | C.7.4.2   | C - Required if time sychronization was applied | 
| Equipment | General Equipment | C.7.5.1 | M |
| Acquisition | General Acquisition | C.7.10.1 | M |
| Image   | General Image     | C.7.6.1   | M |
|         | General Reference | C.12.4    | U |
|         | Image Plane       | C.7.6.2   | M |
|         | Image Pixel       | C.7.6.3   | M |
|         | Contrast/Bolus    | C.7.6.4   | C - Required if contrast was used |
|         | Device            | C.7.6.12  | U |
|         | Specimen          | C.7.6.22  | U |
|         | PET Image          | C.8.9.4   | M |
|         | Overlay Plane     | C.9.2     | U |
|         | VOI LUT           | C.11.2    | U |
|         | Acquisition Context | C.7.6.14 | U |
|         | SOP Common        | C.12.1    | M |
|         | Common Instance Reference | C.12.2 | U |

Legend: M - Mandatory; C - Conditional; U - User Option

### Table A.18.3-1. RT Dose IOD Modules

| IE      | MODULE            | REFERENCE | USAGE |
|---------|-------------------|-----------|---|
| Patient | Patient           | C.7.1.1   | M |
|         | Clinical Trial Subject | C.7.1.3 | U |
| Study   | General Study     | C.7.2.1   | M |
|         | Patient Study     | C.7.2.2   | U |
|         | Clinical Trial Study | C.7.2.3 | U |
| Series  | RT Series         | C.8.8.1   | M |
|         | Clinical Trial Series | C.7.3.2 | U |
| Frame of Reference | Frame of Reference | C.7.4.1 | M |
| Equipment | General Equipment | C.7.5.1 | M |
| Dose    | General Image     | C.7.6.1   | C - Required if dose data contains grid-based doses. |
|         | General Reference | C.12.4    | C - Required if dose data contains grid-based doses. |
|         | Image Plane       | C.7.6.2   | C - Required if dose data contains grid-based doses. |
|         | Image Pixel       | C.7.6.3   | C - Required if dose data contains grid-based doses. |
|         | Multi-frame       | C.7.6.6   | C - Required if dose data contains grid-based doses and pixel data is multi-frame data. |
|         | Overlay Plane     | C.9.2     | U |
|         | Multi-frame Overlay | C.9.3   | U |
|         | Modality LUT      | C.11.1    | U |
|         | RT Dose           | C.8.8.3   | M |
|         | RT DVH            | C.8.8.4   | U |
|         | Structure Set     | C.8.8.5   | C - Required if dose data contains dose points or isodose curves. |
|         | ROI Contour       | C.8.8.6   | C - Required if dose data contains dose points or isodose curves. |
|         | RT Dose ROI       | C.8.8.7   | C - Required if dose data contains dose points or isodose curves. |
|         | SOP Common        | C.12.1    | M |
|         | Common Instance Reference | C.12.2 | U |
|         | Frame Extraction  | C.12.3    | C - Required if the SOP Instance was created in response to a Frame-Level retrieve request. |

Legend: M - Mandatory; C - Conditional; U - User Option

### Table A.19.3-1. RT Structure Set IOD Modules

| IE      | MODULE            | REFERENCE | USAGE |
|---------|-------------------|-----------|---|
| Patient | Patient           | C.7.1.1   | M |
|         | Clinical Trial Subject | C.7.1.3 | U |
| Study   | General Study     | C.7.2.1   | M |
|         | Patient Study     | C.7.2.2   | U |
|         | Clinical Trial Study | C.7.2.3 | U |
| Series  | RT Series         | C.8.8.1   | M |
|         | Clinical Trial Series | C.7.3.2 | U |
| Frame of Reference | Frame of Reference | C.7.4.1 | U |
| Equipment | General Equipment | C.7.5.1 | M |
| Structure Set | Structure Set | C.8.8.5 | M |
|         | ROI Contour       | C.8.8.6   | M |
|         | RT ROI Observations | C.8.8.8 | M |
|         | Approval          | C.8.8.16  | U |
|         | General Reference | C.12.4    | U |
|         | SOP Common        | C.12.1    | M |
|         | Common Instance Reference | C.12.2 | U |

Legend: M - Mandatory; C - Conditional; U - User Option

### Table A.20.3-1. RT Plan IOD Modules

| IE      | MODULE            | REFERENCE | USAGE |
|---------|-------------------|-----------|---|
| Patient | Patient           | C.7.1.1   | M |
|         | Clinical Trial Subject | C.7.1.3 | U |
| Study   | General Study     | C.7.2.1   | M |
|         | Patient Study     | C.7.2.2   | U |
|         | Clinical Trial Study | C.7.2.3 | U |
| Series  | RT Series         | C.8.8.1   | M |
|         | Clinical Trial Series | C.7.3.2 | U |
| Frame of Reference | Frame of Reference | C.7.4.1 | U |
| Equipment | General Equipment | C.7.5.1 | M |
| Plan    | RT General Plan   | C.8.8.9   | M |
|         | RT Prescription   | C.8.8.10  | U |
|         | RT Tolerance Tables | C.8.8.11 | U |
|         | RT Patient Setup  | C.8.8.12  | U |
|         | RT Fraction Scheme | C.8.8.13 | U |
|         | RT Beams          | C.8.8.14  | C - Required if RT Fraction Scheme Module exists and Number of Beams (300A,0080) is greater than zero for one or more fraction groups. |
|         | RT Brachy Application Setups | C.8.8.15 | C - Required if RT Fraction Scheme Module exists and Number of Brachy Application Setups (300A, 00A0) is greater than zero for one or more fraction groups. Incompatibile with RT Beams. |
|         | Approval          | C.8.8.16  | U |
|         | General Reference | C.12.4    | U |
|         | SOP Common        | C.12.1    | M |
|         | Common Instance Reference | C.12.2 | U |

Legend: M - Mandatory; C - Conditional; U - User Option